In [111]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
import gc
import random
import warnings


import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS


import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


import torch
from torch import nn
import torch.optim as optim
from torch.cuda import empty_cache
from torch.utils.data import DataLoader, random_split


from transformers import logging
from transformers import EarlyStoppingCallback
from transformers import TrainingArguments, Trainer
from transformers import BertModel,BertTokenizer


from sklearn.model_selection import train_test_split
from sklearn.decomposition import LatentDirichletAllocation as LDA
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score,classification_report

# from google.colab import drive

In [40]:
warnings.filterwarnings("ignore") # general
logging.set_verbosity_error() # huggingface

In [41]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\itama\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\itama\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\itama\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [119]:
df = pd.read_csv('data/MSRP/msr-para-train.tsv', sep='\t', header=0, error_bad_lines=False, warn_bad_lines=False, names=['label','q1_id','q2_id','q1','q2'],usecols=[0,3,4]).dropna()

In [ ]:
[sen1, sen2, label]

In [120]:
tokenizer = BertTokenizer.from_pretrained('distilbert-base-uncased')
def tokenize_function(s1: str, s2:str, max_length=128, truncation=True):
    return tokenizer(s1, s2, padding='max_length', max_length = max_length, truncation=truncation, return_tensors='pt')

In [135]:
tokenized_data = tokenize_function(df.q1.tolist(), df.q2.tolist())

In [136]:
# ds = TokenizedDataset(tokenized_data, df['label'].tolist())
# dl = DataLoader(ds, batch_size=8)
# for batch in dl:
#     print(batch['input_ids'])

In [147]:
tokenizer('vova', 'asda')

{'input_ids': [101, 29536, 3567, 102, 2004, 2850, 102], 'token_type_ids': [0, 0, 0, 0, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [148]:
from torch.utils.data import DataLoader
from dataset import TokenizedDataset

ds = TokenizedDataset(tokenized_data, df['label'].tolist())

train_dataloader = DataLoader(ds, shuffle=True, batch_size=8)

In [149]:
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

{'input_ids': torch.Size([8, 128]),
 'token_type_ids': torch.Size([8, 128]),
 'attention_mask': torch.Size([8, 128]),
 'labels': torch.Size([8])}

In [155]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [156]:
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

tensor(0.8023, grad_fn=<NllLossBackward0>) torch.Size([8, 2])
